In [ ]:
import numpy as np
import pandas as pd

import xarray as xr
import matplotlib.pyplot as plt

import os 
import time

from renewable_data_load import *


# Configure dask for parallel execution
#import dask
#dask.config.set(scheduler='threads', num_workers=10)  # Adjust num_workers to your CPU cores

/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/intake_esm/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [2]:
# ds_doy.isel(x=110,y=230).plot(x='year',y='dayofyear', cmap='viridis')
# drought_ds.isel(x=110,y=230).plot(x='year',y='dayofyear', cmap='viridis')

In [2]:
resource = "pv"
module = "utility"

simulation = "miroc6"
reference_gwl = 0.8
future_gwl = 2.0

simulations = ["mpi-esm1-2-hr", "miroc6", "taiesm1", 'ec-earth3',]
#simulations = ['ec-earth3']


In [3]:

domain = "d02"
variable = "cf"
frequency = "day"

threshold_file = f"../data/thresholds/{resource}_{module}_{domain}_{variable}_{simulation}_gwlref{reference_gwl}_10th_pctile.nc"
drought_threshold_ds = xr.open_dataset(threshold_file)
ren_ds = get_ren_data_concat(resource, module, domain, variable, frequency, simulation)
ren_ds = ren_ds.convert_calendar("noleap")

# reshape array
ds_doy = ren_ds.copy(deep=True)
ds_doy['dayofyear'] = ds_doy.time.dt.dayofyear
ds_doy['year'] = ds_doy.time.dt.year
ds_doy = ds_doy.assign_coords(
    {"dayofyear":ds_doy.time.dt.dayofyear,
    "year":ds_doy.time.dt.year}
)
# reshape time dimension
ds_doy = ds_doy.drop_vars("time").set_index(time=['dayofyear','year']).unstack()

drought_ds = (ds_doy - drought_threshold_ds.reference_gen).load()

# reshape back into daily timeseries
drought_ds = drought_ds.stack(time=['year','dayofyear'])
drought_ds = drought_ds.reset_index("time").assign_coords(time=ren_ds.time)

drought_ds = drought_ds.load()

# Create binary drought mask for entire time series
# 1 where drought_ds < 0 (below threshold), 0 otherwise
drought_mask = xr.where(drought_ds < 0, 1, 0)
drought_mask.name = "drought_mask"

# save into the drought mask folder

drought_mask = drought_mask.load()

# Save drought mask to file
mask_output_file = f"../data/drought_masks/{resource}_{module}_{domain}_{variable}_{simulation}_gwlref{reference_gwl}_drought_mask_only_full_time.nc"

# Add metadata to drought mask
drought_mask.attrs['resource'] = resource
drought_mask.attrs['module'] = module
drought_mask.attrs['domain'] = domain
drought_mask.attrs['variable'] = variable
drought_mask.attrs['simulation'] = simulation
drought_mask.attrs['reference_gwl'] = float(reference_gwl)
drought_mask.attrs['description'] = 'Binary drought mask: 1 = drought (below threshold), 0 = no drought'

# Save with appropriate encoding
encoding = {'drought_mask': {'dtype': 'int32', '_FillValue': -999}}
drought_mask.to_netcdf(mask_output_file, encoding=encoding, format='NETCDF4')

print(f"Saved drought mask to: {mask_output_file}")
print(f"File size: {os.path.getsize(mask_output_file) / 1e6:.2f} MB")

: 

In [3]:
# calculate drought mask for a single GWL period

reference_gwl = 0.8

mask_gwl = future_gwl

# Get bounds for reference GWL period
WRF_sim_name = sim_name_dict[simulation]
model = WRF_sim_name.split("_")[1]
ensemble_member = WRF_sim_name.split("_")[2]
ref_start_year, ref_end_year = get_gwl_crossing_period(model, ensemble_member, mask_gwl)

domain = "d02"
variable = "cf"
frequency = "day"

threshold_file = f"../data/thresholds/{resource}_{module}_{domain}_{variable}_{simulation}_gwlref{reference_gwl}_10th_pctile.nc"
drought_threshold_ds = xr.open_dataset(threshold_file)
ren_ds = get_ren_data_concat(resource, module, domain, variable, frequency, simulation)
ren_ds = ren_ds.convert_calendar("noleap")


ren_ds = ren_ds.sel(time=slice(f"{ref_start_year}-01-01",f"{ref_end_year}-12-31"))

# reshape array
ds_doy = ren_ds.copy(deep=True)
ds_doy['dayofyear'] = ds_doy.time.dt.dayofyear
ds_doy['year'] = ds_doy.time.dt.year
ds_doy = ds_doy.assign_coords(
    {"dayofyear":ds_doy.time.dt.dayofyear,
    "year":ds_doy.time.dt.year}
)
# reshape time dimension
ds_doy = ds_doy.drop_vars("time").set_index(time=['dayofyear','year']).unstack()

drought_ds = (ds_doy - drought_threshold_ds.reference_gen).load()

# reshape back into daily timeseries
drought_ds = drought_ds.stack(time=['year','dayofyear'])
drought_ds = drought_ds.reset_index("time").assign_coords(time=ren_ds.time)

drought_ds = drought_ds.load()

# Create binary drought mask for entire time series
# 1 where drought_ds < 0 (below threshold), 0 otherwise
drought_mask = xr.where(drought_ds < 0, 1, 0)
drought_mask.name = "drought_mask"

# save into the drought mask folder

drought_mask = drought_mask.load()

# Save drought mask to file
mask_output_file = f"../data/drought_masks/{resource}_{module}_{domain}_{variable}_{simulation}_gwl{mask_gwl}_drought_mask_only.nc"

# Add metadata to drought mask
drought_mask.attrs['resource'] = resource
drought_mask.attrs['module'] = module
drought_mask.attrs['domain'] = domain
drought_mask.attrs['variable'] = variable
drought_mask.attrs['simulation'] = simulation
drought_mask.attrs['reference_gwl'] = float(reference_gwl)
drought_mask.attrs['description'] = 'Binary drought mask: 1 = drought (below threshold), 0 = no drought'

# Save with appropriate encoding
encoding = {'drought_mask': {'dtype': 'int32', '_FillValue': -999}}
drought_mask.to_netcdf(mask_output_file, encoding=encoding, format='NETCDF4')

print(f"Saved drought mask to: {mask_output_file}")
print(f"File size: {os.path.getsize(mask_output_file) / 1e6:.2f} MB")

Saved drought mask to: ../data/drought_masks/pv_utility_d02_cf_miroc6_gwl2.0_drought_mask_only.nc
File size: 3505.24 MB


## Derive drought qualities from drought mask

Calculate three drought event metrics:
1. **Drought duration**: Number of consecutive drought days for each event (stored at first day of event)
2. **Max intensity**: Minimum value during each drought event (most severe deficit)
3. **Magnitude**: Sum of all values during each drought event (cumulative deficit)

In [ ]:
# Get bounds for reference GWL period
WRF_sim_name = sim_name_dict[simulation]
model = WRF_sim_name.split("_")[1]
ensemble_member = WRF_sim_name.split("_")[2]
ref_start_year, ref_end_year = get_gwl_crossing_period(model, ensemble_member, reference_gwl)
fut_start_year, fut_end_year = get_gwl_crossing_period(model, ensemble_member, future_gwl)

### Configure time chunking parameters

In [35]:
drought_metrics

<xarray.Dataset> Size: 12GB
Dimensions:            (y: 320, x: 250, time: 7300)
Coordinates:
    year               (time) int32 29kB 1981 1981 1981 1981 ... 2000 2000 2000
    dayofyear          (time) int32 29kB 1 2 3 4 5 6 ... 360 361 362 363 364 365
  * time               (time) object 58kB 1981-01-01 00:00:00 ... 2000-12-31 ...
    landmask           (y, x) float32 320kB 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0
    lat                (y, x) float32 320kB 23.35 23.39 23.43 ... 56.42 56.46
    lon                (y, x) float32 320kB -116.5 -116.4 ... -111.6 -111.5
  * y                  (y) float64 3kB -2.441e+05 -2.351e+05 ... 2.627e+06
  * x                  (x) float64 2kB -4.638e+06 -4.629e+06 ... -2.397e+06
    Lambert_Conformal  int32 4B 1
    quantile           float64 8B 0.1
Data variables:
    duration           (y, x, time) float32 2GB nan nan nan nan ... nan nan nan
    intensity          (y, x, time) float32 2GB nan nan nan nan ... nan nan nan
    magnitude          (y, x, time) float32 2GB nan nan nan nan ... nan nan nan
    mask               (y, x, time) int64 5GB 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
Attributes:
    resource:       pv
    module:         utility
    domain:         d02
    variable:       cf
    simulation:     taiesm1
    reference_gwl:  0.8
    description:    Drought event metrics: duration, intensity (min value), a...

In [29]:
drought_metrics.attrs.clear()

In [37]:
# Save the merged drought metrics with detailed metadata
output_file = f"../data/drought_masks/{resource}_{module}_{domain}_{variable}_{simulation}_gwlref{reference_gwl}_drought_mask_metrics.nc"

# Explicitly set encoding for all variables to avoid int64 overflow issues
encoding = {
    'duration': {'dtype': 'float32', '_FillValue': np.nan},
    'intensity': {'dtype': 'float32', '_FillValue': np.nan},
    'magnitude': {'dtype': 'float32', '_FillValue': np.nan},
    'mask': {'dtype': 'int32'}  # Convert int64 to int32 for NetCDF3 compatibility
}

print(f"Saving drought metrics to: {output_file}")
print(f"Total processing time: {total_elapsed:.2f} seconds ({total_elapsed/60:.2f} minutes)")

# Use NetCDF4 format to avoid int32 limitations
drought_metrics.to_netcdf(output_file, encoding=encoding, format='NETCDF4')

print(f"Successfully saved drought metrics!")
print(f"\nFile: {output_file}")
print(f"Size: {os.path.getsize(output_file) / 1e9:.2f} GB")


Saving drought metrics to: ../data/drought_masks/pv_utility_d02_cf_taiesm1_gwlref0.8_drought_mask_metrics.nc
Total processing time: 193.07 seconds (3.22 minutes)
Successfully saved drought metrics!

File: ../data/drought_masks/pv_utility_d02_cf_taiesm1_gwlref0.8_drought_mask_metrics.nc
Size: 9.35 GB
Successfully saved drought metrics!

File: ../data/drought_masks/pv_utility_d02_cf_taiesm1_gwlref0.8_drought_mask_metrics.nc
Size: 9.35 GB


### Optional: Clean up temporary chunk files

## Interactive spatial visualization of drought mask

In [ ]:
# from ipywidgets import interact, IntSlider
# import matplotlib.pyplot as plt

# def plot_drought_mask_timestep(time_idx):
#     """Plot the spatial drought mask for a given time index"""
#     fig, ax = plt.subplots(figsize=(10, 8))
    
#     # Get the drought mask at this time step
#     mask_slice = drought_mask.isel(time=time_idx)
    
#     # Plot
#     im = ax.pcolormesh(
#         mask_slice.x, 
#         mask_slice.y, 
#         mask_slice.values,
#         cmap='RdYlBu_r',
#         vmin=0,
#         vmax=1,
#         shading='auto'
#     )
    
#     # Get the time value
#     time_val = drought_mask.time.values[time_idx]
    
#     ax.set_title(f'Drought Mask - Time: {time_val}', fontsize=14)
#     ax.set_xlabel('X coordinate')
#     ax.set_ylabel('Y coordinate')
#     ax.set_aspect('equal')
    
#     # Add colorbar
#     cbar = plt.colorbar(im, ax=ax)
#     cbar.set_label('Drought (1) / No Drought (0)', rotation=270, labelpad=20)
    
#     plt.tight_layout()
#     plt.show()

# # Create interactive slider
# n_times = len(drought_mask.time)
# time_slider = IntSlider(
#     min=0, 
#     max=n_times-1, 
#     step=1, 
#     value=0,
#     description='Time step:',
#     continuous_update=False
# )

# interact(plot_drought_mask_timestep, time_idx=time_slider)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Time step:', max=43069), Output…

<function __main__.plot_drought_mask_timestep(time_idx)>